<a href="https://colab.research.google.com/github/harveenchadha/TTS/blob/main/notebooks/vakyansh_tts_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing Dependencies

In [1]:
import os
!git clone https://github.com/Open-Speech-EkStep/vakyansh-tts
os.chdir('vakyansh-tts') 
!bash install.sh
!python setup.py bdist_wheel
!pip install -e .
os.chdir('tts_infer')
!mkdir translit_models
os.chdir('translit_models')
!wget https://storage.googleapis.com/vakyaansh-open-models/translit_models/default_lineup.json
!mkdir hindi
os.chdir('hindi')
!wget https://storage.googleapis.com/vakyaansh-open-models/translit_models/hindi/hindi_transliteration.zip
!unzip hindi_transliteration

!wget https://storage.googleapis.com/vakyansh-open-models/tts/hindi/hi-IN/female_voice_0/glow.zip
!unzip glow.zip

!wget https://storage.googleapis.com/vakyansh-open-models/tts/hindi/hi-IN/female_voice_0/hifi.zip
!unzip hifi.zip

!rm glow.zip
!rm hifi.zip

os.chdir('/content/vakyansh-tts/')

Cloning into 'vakyansh-tts'...
remote: Enumerating objects: 466, done.
remote: Counting objects: 100% (201/201), done.
remote: Compressing objects: 100% (175/175), done.
remote: Total 466 (delta 89), reused 64 (delta 22), pack-reused 265
Receiving objects: 100% (466/466), 259.27 KiB | 1.39 MiB/s, done.
Resolving deltas: 100% (229/229), done.
Processing /content/vakyansh-tts/src/glow_tts/monotonic_align
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for monotonic-align: filename=monotonic_align-1.1-cp37-c

--2022-01-04 08:20:03--  https://storage.googleapis.com/vakyaansh-open-models/translit_models/default_lineup.json
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.128, 2607:f8b0:4023:c0b::80
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3422 (3.3K) [application/json]
Saving to: ‘default_lineup.json’

default_lineup.json 100%[===================>]   3.34K  --.-KB/s    in 0s      

2022-01-04 08:20:03 (44.3 MB/s) - ‘default_lineup.json’ saved [3422/3422]

--2022-01-04 08:20:03--  https://storage.googleapis.com/vakyaansh-open-models/translit_models/hindi/hindi_transliteration.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.128, 2607:f8b0:4023:c0b::80
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45018357 (43M) [application/zip]


## Inference Code

In [2]:
from tts_infer.tts import TextToMel, MelToWav
from tts_infer.transliterate import XlitEngine
from tts_infer.num_to_word_on_sent import normalize_nums

import re
from scipy.io.wavfile import write
device = 'cpu'

text_to_mel = TextToMel(glow_model_dir='/content/vakyansh-tts/tts_infer/translit_models/hindi/glow_ckp', device=device)
mel_to_wav = MelToWav(hifi_model_dir='/content/vakyansh-tts/tts_infer/translit_models/hindi/hifi_ckp', device=device)

def translit(text, lang):
    reg = re.compile(r'[a-zA-Z]')
    engine = XlitEngine(lang)
    words = [engine.translit_word(word, topk=1)[lang][0] if reg.match(word) else word for word in text.split()]
    updated_sent = ' '.join(words)
    return updated_sent
    
def run_tts(text, lang):
    text = text.replace('।', '.') # only for hindi models
    text_num_to_word = normalize_nums(text, lang) # converting numbers to words in lang
    text_num_to_word_and_transliterated = translit(text_num_to_word, lang) # transliterating english words to lang
    
    mel = text_to_mel.generate_mel(text_num_to_word_and_transliterated)
    audio, sr = mel_to_wav.generate_wav(mel)
    write(filename='temp.wav', rate=sr, data=audio) # for saving wav file, if needed
    return (sr, audio)

/content/vakyansh-tts/tts_infer/translit_models/hindi/glow_ckp/G_250.pth
INFO:root:Loaded checkpoint '/content/vakyansh-tts/tts_infer/translit_models/hindi/glow_ckp/G_250.pth' (iteration 250)
/content/vakyansh-tts/tts_infer/translit_models/hindi/hifi_ckp/g_00100000
Loading '/content/vakyansh-tts/tts_infer/translit_models/hindi/hifi_ckp/g_00100000'
Complete.
Removing weight norm...


In [3]:
_, audio = run_tts('hello my name is harveen', 'hi')

Loading hi...


## Results

In [4]:
import IPython.display as ipd
ipd.Audio('temp.wav')